 # `AVDOS-VR` - Virtual Reality Affective Video Database with Physiological Signals

Notebook containing the scripts to verify the data and summarize the sessions for the AVDOS-VR dataset.

It analyses the events (.json) and physiological responses (.csv) per participant, and produces a set of files: 

1) Signal quality per participant, grouped per stages: slow_movement, fast_movement, and videos.
2) Plot synchronizing PPG reponses and events, grouped per stages (slow_movement, fast_movement, videos 1 to 5), 

In [1]:
from pathlib import Path
import sys,os
this_path = None
try:    # WORKS WITH .py
    this_path = str(os.path.dirname(os.path.abspath(__file__)))
except: # WORKS WITH .ipynb
    this_path = str(Path().absolute())+"/" 
print("File Path:", this_path)

# Add the level up to the file path so it recognizes the scripts inside `avdos-vr`
sys.path.append(os.path.join(this_path, ".."))

File Path: e:\dsv\dev\git_repos\AVDOS-VR\notebooks/


In [2]:
import os
import numpy as np
import pandas as pd

import avdosvr
from avdosvr.verify_data import verify_data
from avdosvr.plots.plots import plot_sm_ppg, plot_fit_state
from avdosvr.utils.classes import Participant
from avdosvr.utils.files_handler import generate_complete_path

from avdosvr.analysis.average_ratings import calculate_average_arousal_valence_ratings, get_average_av_ratings_columns
from avdosvr.analysis.durations import calculate_good_signal_quality_duration, calculate_signal_quality_check_duration, get_durations_columns
from avdosvr.analysis.skip_participants import skip_participant

import avdosvr.hrv.HRV_analysis_cloud as HRV

# Hide warning about pandas Slicing
pd.options.mode.chained_assignment = None


In [3]:
#%% Set variables

#Directory containing participant data downloaded from gnacek.com/affective-video-database-online-study
data_directory = (r"../data")
# Identifier for the notebook when creating temp files
NOTEBOOK_NAME = "0_verify/" 

#Bool variables to enable/disable processing of segments of the data to speed up the runtime
process_slow_movement = False
process_fast_movement = False
process_video = True

validate_data = True

calculate_duration = True
calculate_average_av_ratings = True

#Bool variables to enable/disable plotting of graphs
plots_output_root_folder = "./temp/"+NOTEBOOK_NAME
plot_slow_movement_ppg = False
plot_contact_fit_state = False

no_of_frames_to_drop_from_start_of_recording = 1001


In [4]:
# Data for each participant is stored within a folder. Names of files indicate which segment of remote study it is
list_data_dir = os.listdir(data_directory)
is_folder = [os.path.isdir(os.path.join(data_directory,d)) for d in list_data_dir]
participants_list = np.array(list_data_dir)[ is_folder ]

#%% Get list of participant folders/files
print("Found data folders for " + str(len(participants_list)) + " participants")

Found data folders for 39 participants


In [5]:
#%% Run data validation check on all participants
if(validate_data):
    fit_states_signal_quality = None
    for participant in participants_list:
        new_df = verify_data(data_directory + "/" + participant)
        fit_states_signal_quality = new_df if (fit_states_signal_quality is None) else pd.concat([fit_states_signal_quality,new_df], axis=0)
    print("----FINISHED DATA CHECK FOR ALL PARTICIPANTS SEE ABOVE FOR OUTPUT----")


 Running data check for:  ../data/participant_101
 Running data check for:  ../data/participant_216
 Running data check for:  ../data/participant_219
 Running data check for:  ../data/participant_222
 Running data check for:  ../data/participant_247
 Running data check for:  ../data/participant_248
 Running data check for:  ../data/participant_268
 Running data check for:  ../data/participant_270
 Running data check for:  ../data/participant_278
 Running data check for:  ../data/participant_290
 Running data check for:  ../data/participant_293
 Running data check for:  ../data/participant_299
 Running data check for:  ../data/participant_307
 Running data check for:  ../data/participant_308
 Running data check for:  ../data/participant_309
 Running data check for:  ../data/participant_310
 Running data check for:  ../data/participant_312
 Running data check for:  ../data/participant_314
 Running data check for:  ../data/participant_321
 Running data check for:  ../data/participant_322


In [6]:
fit_states_signal_quality

,participant_number,video_movement_signal_quality,protocol
0,216,9,v1
0,216,9,v1
0,219,9,v1
0,222,9,v1
0,247,9,v1
0,248,9,v1
0,268,9,v1
0,270,9,v1
0,278,9,v1
0,290,9,v1


In [7]:
# Save
fit_states_signal_quality.to_csv( generate_complete_path("fit_states_signal_quality", subfolders=NOTEBOOK_NAME, file_extension=".csv") )

In [8]:
def drop_start_frames(data, frames_to_drop):
    return data.drop(data.index[0:frames_to_drop])

In [ ]:
#%% Main analysis script

if(calculate_duration):
    durations = pd.DataFrame(columns = get_durations_columns())
if(calculate_average_av_ratings):
     average_av_ratings = pd.DataFrame(columns = get_average_av_ratings_columns())

participant_counter = 1
# debug_counter = 1

#Loop through all participant data
for participant in participants_list:
    # Processing slow movement data
    # if(participant_counter==1):
    #     participant = participants_list[debug_counter]
    # else:
    #     participant = participants_list[debug_counter+participant_counter]
    ParticipantObj = Participant(participant, data_directory)

    print("Processing data for: " + ParticipantObj.name + ". " + str(participant_counter) + " out of " + str(len(participants_list)))
    ParticipantObj = skip_participant(ParticipantObj)
    
    if process_slow_movement:
        if(ParticipantObj.skip_sm == False):
            print("Slow movement Start")
            slowMovementData = ParticipantObj.getSlowMovementData()
            sm_events = slowMovementData[slowMovementData['Event'] != '']
            slowMovementData = drop_start_frames(slowMovementData, no_of_frames_to_drop_from_start_of_recording) #drop first 1000 frames/bug with timestamps
            if(calculate_duration):
                slowMovementDuration = round((slowMovementData['Time'].iloc[-1] - slowMovementData['Time'].iloc[0]),3)
                slowMovementSignalCheckDuration = calculate_signal_quality_check_duration(sm_events)
                slowMovementGoodDuration, slowMovementTimeTakenToEstablishGoodSignal = calculate_good_signal_quality_duration(slowMovementData)
                slowMovementDataExcludingSignalCheck = slowMovementData.drop(slowMovementData.index[0:sm_events['Frame#'].iloc[1]-1])
                slowMovementDurationExcludingSignalCheck = round((slowMovementDataExcludingSignalCheck['Time'].iloc[-1] - slowMovementDataExcludingSignalCheck['Time'].iloc[0]),3)
                slowMovementGoodDurationExcludingSignalCheck, blank = calculate_good_signal_quality_duration(slowMovementData.drop(slowMovementData.index[0:sm_events['Frame#'].iloc[1]-1])) #drop all rows up until the signal check is finished
            
    
            
            signal_to_analyze = slowMovementData.iloc[::20,:] #convert 1000Hz to 50 for IMU
            signal_to_plot = signal_to_analyze[::2] #ppg signal is 25Hz, take every 2nd 
            signal_to_analyze = signal_to_plot['Ppg/Raw.ppg'] #ppg signal is 25Hz, take every 2nd 
            signal_to_analyze.reset_index(inplace=True, drop=True)  
            signal_to_analyze = signal_to_analyze.astype(float)
            # feats_ppg,filtered_sensor_data,rr,timings,peak_indx, = HRV.get_HRV_features(signal_to_analyze,
            #                                                                             ma=True,  
            #                                                                             detrend = False,     
            #                                                                             band_pass=True, 
            #                                                                             thres=.75,
            #                                                                             winsorize=True,
            #                                                                             winsorize_value=5,
            #                                                                             dynamic_threshold=True,  
            #                                                                             dynamic_threshold_value=1,
            #                                                                             hampel_fiter=True,
            #                                                                             sampling=25)
            
    
            if plot_slow_movement_ppg:
                plot_sm_ppg(signal_to_plot, sm_events, ParticipantObj.name, root_folder=plots_output_root_folder)
            if plot_contact_fit_state:
                plot_fit_state(slowMovementData, sm_events, ParticipantObj.name, "Slow Movement", root_folder=plots_output_root_folder)
            print("Slow movement End")
        else:
            print("Skipping SM for:" + ParticipantObj.name)
            slowMovementDuration = "SKIPPED" 
            slowMovementGoodDuration = "SKIPPED" 
            slowMovementTimeTakenToEstablishGoodSignal = "SKIPPED"
            slowMovementSignalCheckDuration = "SKIPPED"
            slowMovementDurationExcludingSignalCheck = "SKIPPED"
            slowMovementGoodDurationExcludingSignalCheck = "SKIPPED"

    if process_fast_movement:
        if(ParticipantObj.skip_fm == False):
            print("Fast movement Start")
            fastMovementData = ParticipantObj.getFastMovementData()
            fm_events = fastMovementData[fastMovementData['Event'] != '']
            fastMovementData = drop_start_frames(fastMovementData, no_of_frames_to_drop_from_start_of_recording) #drop first 500 frames/bug with timestamps
            if(calculate_duration):
                fastMovementDuration = round((fastMovementData['Time'].iloc[-1] - fastMovementData['Time'].iloc[0]),3)
                fastMovementGoodDuration, fastMovementTimeTakenToEstablishGoodSignal = calculate_good_signal_quality_duration(fastMovementData)
                fastMovementSignalCheckDuration = calculate_signal_quality_check_duration(fm_events)
                fastMovementDataExcludingSignalCheck = fastMovementData.drop(fastMovementData.index[0:fm_events['Frame#'].iloc[1]-1])
                fastMovementDurationExcludingSignalCheck = round((fastMovementDataExcludingSignalCheck['Time'].iloc[-1] - fastMovementDataExcludingSignalCheck['Time'].iloc[0]),3)
                fastMovementGoodDurationExcludingSignalCheck, blank = calculate_good_signal_quality_duration(fastMovementData.drop(fastMovementData.index[0:fm_events['Frame#'].iloc[1]-1])) #drop 
            if plot_contact_fit_state:
                plot_fit_state(fastMovementData, fm_events, ParticipantObj.name, "Fast Movement", root_folder=plots_output_root_folder)
            print("Fast movement End")
        else:
            print("Skipping FM for:" + ParticipantObj.name)
            fastMovementDuration = "SKIPPED" 
            fastMovementGoodDuration = "SKIPPED" 
            fastMovementTimeTakenToEstablishGoodSignal = "SKIPPED" 
            fastMovementSignalCheckDuration = "SKIPPED" 
            fastMovementDurationExcludingSignalCheck = "SKIPPED" 
            fastMovementGoodDurationExcludingSignalCheck = "SKIPPED" 

    if process_video:
        if(ParticipantObj.skip_video == False):
            print("Video Start")
            if(ParticipantObj.skip_video_1 == False):
                video_1_data = ParticipantObj.getVideo1Data()
                video_1_events = video_1_data[video_1_data['Event'] != '']
                video_1_data = drop_start_frames(video_1_data, no_of_frames_to_drop_from_start_of_recording)
                if(calculate_duration):
                    video_1_duration = round((video_1_data['Time'].iloc[-1] - video_1_data['Time'].iloc[0]),3)
                    video_1_GoodDuration, video_1_TimeTakenToEstablishGoodSignal = calculate_good_signal_quality_duration(video_1_data)
                    video_1_SignalCheckDuration = calculate_signal_quality_check_duration(video_1_events)
                    video_1_DataExcludingSignalCheck = video_1_data.drop(video_1_data.index[0:video_1_events['Frame#'].iloc[1]-1])
                    video_1_DurationExcludingSignalCheck = round((video_1_DataExcludingSignalCheck['Time'].iloc[-1] - video_1_DataExcludingSignalCheck['Time'].iloc[0]),3)
                    video_1_GoodDurationExcludingSignalCheck, blank = calculate_good_signal_quality_duration(video_1_data.drop(video_1_data.index[0:video_1_events['Frame#'].iloc[1]-1])) #drop 
                if plot_contact_fit_state:
                    plot_fit_state(video_1_data, video_1_events, ParticipantObj.name, "Video 1", root_folder=plots_output_root_folder)
            else:
                print("Skipping VIDEO_1 for:" + ParticipantObj.name)
                video_1_duration = "SKIPPED" 
                video_1_GoodDuration = "SKIPPED"  
                video_1_TimeTakenToEstablishGoodSignal = "SKIPPED" 
                video_1_SignalCheckDuration = "SKIPPED" 
                video_1_DurationExcludingSignalCheck = "SKIPPED" 
                video_1_GoodDurationExcludingSignalCheck = "SKIPPED"
                
            if(ParticipantObj.skip_video_2 == False):
                video_2_data = ParticipantObj.getVideo2Data()
                video_2_events = video_2_data[video_2_data['Event'] != '']
                if(calculate_average_av_ratings):
                    video_2_ratings = calculate_average_arousal_valence_ratings(video_2_events)
                video_2_data = drop_start_frames(video_2_data, no_of_frames_to_drop_from_start_of_recording)
                if(calculate_duration):
                    video_2_duration = video_2_data['Time'].iloc[-1] - video_2_data['Time'].iloc[0]
                    video_2_GoodDuration, blank = calculate_good_signal_quality_duration(video_2_data)
                if plot_contact_fit_state:
                    plot_fit_state(video_2_data, video_2_events, ParticipantObj.name, "Video 2", root_folder=plots_output_root_folder)
            else:
                print("Skipping VIDEO_2 for:" + ParticipantObj.name)
                video_2_duration = "SKIPPED" 
                video_2_GoodDuration = "SKIPPED" 
                
            if(ParticipantObj.skip_video_3 == False):
                video_3_data = ParticipantObj.getVideo3Data()
                video_3_events = video_3_data[video_3_data['Event'] != '']
                if(calculate_average_av_ratings):
                    video_3_ratings = calculate_average_arousal_valence_ratings(video_3_events)
                video_3_data = drop_start_frames(video_3_data, no_of_frames_to_drop_from_start_of_recording)
                if(calculate_duration):
                    video_3_duration = video_3_data['Time'].iloc[-1] - video_3_data['Time'].iloc[0]
                    video_3_GoodDuration, blank = calculate_good_signal_quality_duration(video_3_data)
                if plot_contact_fit_state:
                    plot_fit_state(video_3_data, video_3_events, ParticipantObj.name, "Video 3", root_folder=plots_output_root_folder)
            else:
                print("Skipping VIDEO_3 for:" + ParticipantObj.name)
                video_3_duration = "SKIPPED" 
                video_3_GoodDuration = "SKIPPED" 
                
            if(ParticipantObj.skip_video_4 == False):
                video_4_data = ParticipantObj.getVideo4Data()
                video_4_events = video_4_data[video_4_data['Event'] != '']
                if(calculate_average_av_ratings):
                    video_4_ratings = calculate_average_arousal_valence_ratings(video_4_events)
                video_4_data = drop_start_frames(video_4_data, no_of_frames_to_drop_from_start_of_recording)
                if(calculate_duration):
                    video_4_duration = video_4_data['Time'].iloc[-1] - video_4_data['Time'].iloc[0]
                    video_4_GoodDuration, blank = calculate_good_signal_quality_duration(video_4_data)
                if plot_contact_fit_state:
                    plot_fit_state(video_4_data, video_4_events, ParticipantObj.name, "Video 4", root_folder=plots_output_root_folder)
            else:
                print("Skipping VIDEO_4 for:" + ParticipantObj.name)
                video_4_duration = "SKIPPED" 
                video_4_GoodDuration = "SKIPPED" 

            if(ParticipantObj.skip_video_5 == False):
                video_5_data = ParticipantObj.getVideo5Data()
                video_5_events = video_5_data[video_5_data['Event'] != '']
                if(calculate_average_av_ratings):
                    video_5_ratings = calculate_average_arousal_valence_ratings(video_5_events)
                video_5_data = drop_start_frames(video_5_data, no_of_frames_to_drop_from_start_of_recording)
                if(calculate_duration):
                    video_5_duration = video_5_data['Time'].iloc[-1] - video_5_data['Time'].iloc[0]
                    video_5_GoodDuration, blank = calculate_good_signal_quality_duration(video_5_data)
                if plot_contact_fit_state:
                    plot_fit_state(video_5_data, video_5_events, ParticipantObj.name, "Video 5", root_folder=plots_output_root_folder)
            else:
                print("Skipping VIDEO_5 for:" + ParticipantObj.name)
                video_5_duration = "SKIPPED" 
                video_5_GoodDuration = "SKIPPED" 
            
            if(calculate_average_av_ratings):
                current_participant_average_av_ratings = {"participant": ParticipantObj.name, 
                                      "relax_2_valence": None, "relax_2_arousal": None, "relax_2_no_ratings": None, "relax_3_valence": None, "relax_3_arousal": None, "relax_3_no_ratings": None, "relax_4_valence": None, "relax_4_arousal": None, "relax_4_no_ratings": None, "relax_5_valence": None, "relax_5_arousal": None, "relax_5_no_ratings": None,
                                      "video_03_valence": None, "video_03_arousal": None, "video_03_no_ratings": None, "video_04_valence": None, "video_04_arousal": None, "video_04_no_ratings": None, "video_05_valence": None, "video_05_arousal": None, "video_05_no_ratings": None, "video_06_valence": None, "video_06_arousal": None, "video_06_no_ratings": None, "video_10_valence": None, "video_10_arousal": None, "video_10_no_ratings": None, "video_12_valence": None, "video_12_arousal": None, "video_12_no_ratings": None, "video_13_valence": None, "video_13_arousal": None, "video_13_no_ratings": None, "video_18_valence": None, "video_18_arousal": None, "video_18_no_ratings": None, "video_19_valence": None, "video_19_arousal": None, "video_19_no_ratings": None, "video_20_valence": None, "video_20_arousal": None, "video_20_no_ratings": None,
                                      "video_21_valence": None, "video_21_arousal": None, "video_21_no_ratings": None, "video_22_valence": None, "video_22_arousal": None, "video_22_no_ratings": None, "video_23_valence": None, "video_23_arousal": None, "video_23_no_ratings": None, "video_25_valence": None, "video_25_arousal": None, "video_25_no_ratings": None, "video_29_valence": None, "video_29_arousal": None, "video_29_no_ratings": None, "video_31_valence": None, "video_31_arousal": None, "video_31_no_ratings": None, "video_33_valence": None, "video_33_arousal": None, "video_33_no_ratings": None, "video_37_valence": None, "video_37_arousal": None, "video_37_no_ratings": None, "video_38_valence": None, "video_38_arousal": None, "video_38_no_ratings": None, "video_39_valence": None, "video_39_arousal": None, "video_39_no_ratings": None,
                                      "video_41_valence": None, "video_41_arousal": None, "video_41_no_ratings": None, "video_42_valence": None, "video_42_arousal": None, "video_42_no_ratings": None, "video_46_valence": None, "video_46_arousal": None, "video_46_no_ratings": None, "video_48_valence": None, "video_48_arousal": None, "video_48_no_ratings": None, "video_49_valence": None, "video_49_arousal": None, "video_49_no_ratings": None, "video_51_valence": None, "video_51_arousal": None, "video_51_no_ratings": None, "video_55_valence": None, "video_55_arousal": None, "video_55_no_ratings": None, "video_56_valence": None, "video_56_arousal": None, "video_56_no_ratings": None, "video_57_valence": None, "video_57_arousal": None, "video_57_no_ratings": None, "video_58_valence": None, "video_58_arousal": None, "video_58_no_ratings": None}  
                current_participant_average_av_ratings['relax_2_valence'] = video_2_ratings[0].average_valence
                current_participant_average_av_ratings['relax_2_arousal'] = video_2_ratings[0].average_arousal
                current_participant_average_av_ratings['relax_2_no_ratings'] = len(video_2_ratings[0].arousal_ratings)
                current_participant_average_av_ratings['relax_3_valence'] = video_3_ratings[0].average_valence
                current_participant_average_av_ratings['relax_3_arousal'] = video_3_ratings[0].average_arousal
                current_participant_average_av_ratings['relax_3_no_ratings'] = len(video_3_ratings[0].arousal_ratings)
                current_participant_average_av_ratings['relax_4_valence'] = video_4_ratings[0].average_valence
                current_participant_average_av_ratings['relax_4_arousal'] = video_4_ratings[0].average_arousal
                current_participant_average_av_ratings['relax_4_no_ratings'] = len(video_4_ratings[0].arousal_ratings)
                current_participant_average_av_ratings['relax_5_valence'] = video_5_ratings[0].average_valence
                current_participant_average_av_ratings['relax_5_arousal'] = video_5_ratings[0].average_arousal
                current_participant_average_av_ratings['relax_5_no_ratings'] = len(video_5_ratings[0].arousal_ratings)
                
                combined_affective_video_ratings = video_2_ratings + video_3_ratings + video_4_ratings
                for video_rating in combined_affective_video_ratings:
                    if(video_rating.name!="relax"):
                        current_participant_average_av_ratings["video_" + video_rating.name + "_valence"] = video_rating.average_valence
                        current_participant_average_av_ratings["video_" + video_rating.name + "_arousal"] = video_rating.average_arousal
                        current_participant_average_av_ratings["video_" + video_rating.name + "_no_ratings"] = len(video_rating.arousal_ratings)

                average_av_ratings = average_av_ratings.append(current_participant_average_av_ratings, ignore_index = True)
                # average_av_ratings = pd.concat([average_av_ratings, current_participant_average_av_ratings], axis=0, ignore_index = True)
                
            print("Video End")
        else:
            print("Skipping VIDEO for:" + ParticipantObj.name)
            video_1_duration = "SKIPPED" 
            video_1_GoodDuration = "SKIPPED"  
            video_1_TimeTakenToEstablishGoodSignal = "SKIPPED" 
            video_1_SignalCheckDuration = "SKIPPED" 
            video_1_DurationExcludingSignalCheck = "SKIPPED" 
            video_1_GoodDurationExcludingSignalCheck = "SKIPPED" 
            video_2_duration = "SKIPPED" 
            video_2_GoodDuration = "SKIPPED" 
            video_3_duration = "SKIPPED" 
            video_3_GoodDuration = "SKIPPED" 
            video_4_duration = "SKIPPED" 
            video_4_GoodDuration = "SKIPPED" 
            video_5_duration = "SKIPPED" 
            video_5_GoodDuration = "SKIPPED" 
    
    if(calculate_duration):
        current_participant_durations = {'participant': ParticipantObj.name,
                                        #  'slow_movement_total_duration': slowMovementDuration,'slow_movement_total_good_fit': slowMovementGoodDuration, 'slowMovementTimeTakenToEstablishGoodSignal': slowMovementTimeTakenToEstablishGoodSignal,'slow_movement_signal_check_duration': slowMovementSignalCheckDuration,'slow_movement_duration_excluding_signal_check': slowMovementDurationExcludingSignalCheck,'slow_movement_good_fit_excluding_signal_check': slowMovementGoodDurationExcludingSignalCheck,
                                        #  'fast_movement_total_duration': fastMovementDuration,'fast_movement_total_good_fit': fastMovementGoodDuration, 'fastMovementTimeTakenToEstablishGoodSignal': fastMovementTimeTakenToEstablishGoodSignal,'fast_movement_signal_check_duration': fastMovementSignalCheckDuration,'fast_movement_duration_excluding_signal_check': fastMovementDurationExcludingSignalCheck,'fast_movement_good_fit_excluding_signal_check': fastMovementGoodDurationExcludingSignalCheck,
                                         'video_1_total_duration': video_1_duration,'video_1_total_good_fit': video_1_GoodDuration, 'video_1_TimeTakenToEstablishGoodSignal': video_1_TimeTakenToEstablishGoodSignal,'video_1_signal_check_duration': video_1_SignalCheckDuration,'video_1_duration_excluding_signal_check': video_1_DurationExcludingSignalCheck,'video_1_good_fit_excluding_signal_check': video_1_GoodDurationExcludingSignalCheck,
                                         'video_2_duration': video_2_duration, 'video_2_good_fit': video_2_GoodDuration,
                                         'video_3_duration': video_3_duration, 'video_3_good_fit': video_3_GoodDuration,
                                         'video_4_duration': video_4_duration, 'video_4_good_fit': video_4_GoodDuration,
                                         'video_5_duration': video_5_duration, 'video_5_good_fit': video_5_GoodDuration}

        durations = durations.append(current_participant_durations, ignore_index = True)
        # durations = pd.concat([durations,current_participant_durations], axis=0, ignore_index = True)
   
    participant_counter = participant_counter + 1
print("Finished processing data for: " + ParticipantObj.name)
    

In [ ]:
average_av_ratings

,participant,relax_2_valence,relax_2_arousal,relax_2_no_ratings,relax_3_valence,relax_3_arousal,relax_3_no_ratings,relax_4_valence,relax_4_arousal,relax_4_no_ratings,...,video_55_no_ratings,video_56_valence,video_56_arousal,video_56_no_ratings,video_57_valence,video_57_arousal,video_57_no_ratings,video_58_valence,video_58_arousal,video_58_no_ratings
0,participant_101,6.633,3.041,49,4.950,2.500,40,7.585,5.780,41,...,13,5.000,9.000,8,5.000,8.538,13,6.389,6.444,18
1,participant_216,4.847,5.102,59,4.476,3.667,21,3.057,7.057,35,...,59,4.074,5.519,27,4.167,6.750,48,6.465,7.419,43
2,participant_219,6.226,1.321,53,4.804,3.435,46,4.950,3.400,20,...,49,6.250,6.450,20,7.056,6.500,18,7.038,6.308,26
3,participant_222,4.387,2.742,31,5.741,4.296,27,4.647,2.706,17,...,14,6.537,6.902,41,6.000,5.722,18,7.500,7.364,22
4,participant_247,6.612,4.490,49,6.583,4.167,24,4.837,2.047,43,...,16,6.147,6.765,34,5.167,3.667,18,5.172,6.379,29
5,participant_248,6.512,2.442,86,7.148,2.025,81,4.103,5.717,145,...,32,5.833,5.111,18,5.448,5.000,29,6.632,5.737,38
6,participant_268,7.694,2.194,36,1.844,5.000,64,3.886,3.955,44,...,28,3.500,7.750,16,5.731,6.885,26,5.762,6.571,21
7,participant_270,8.302,1.774,53,4.766,3.947,94,7.143,1.000,7,...,11,8.538,5.077,13,8.200,5.400,5,9.000,4.000,1
8,participant_278,6.947,3.526,19,5.600,2.600,15,4.350,2.500,20,...,9,8.091,8.636,11,7.778,7.444,9,4.917,5.667,12
9,participant_290,7.664,3.565,131,7.323,5.000,62,6.921,1.816,38,...,34,7.439,7.244,41,6.682,6.864,22,7.885,6.231,26


In [ ]:
durations

,participant,slow_movement_total_duration,slow_movement_total_good_fit,slowMovementTimeTakenToEstablishGoodSignal,slow_movement_signal_check_duration,slow_movement_duration_excluding_signal_check,slow_movement_good_fit_excluding_signal_check,fast_movement_total_duration,fast_movement_total_good_fit,fastMovementTimeTakenToEstablishGoodSignal,...,video_1_duration_excluding_signal_check,video_1_good_fit_excluding_signal_check,video_2_duration,video_2_good_fit,video_3_duration,video_3_good_fit,video_4_duration,video_4_good_fit,video_5_duration,video_5_good_fit
0,participant_101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,159.508,159.508,426.25134,425.139,426.38598,423.228,426.42993,424.198,122.51307,121.426
1,participant_216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,141.725,141.725,418.50501,417.446,418.44604,417.381,418.469,418.469,118.60803,115.455
2,participant_219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,266.723,266.723,424.90771,423.841,425.43994,424.375,425.76514,424.687,121.18262,120.113
3,participant_222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,219.959,218.272,424.13098,423.062,425.0741,421.731,425.68176,423.303,122.38782,121.321
4,participant_247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,262.59,262.59,422.90943,421.856,423.2594,422.209,423.52722,413.585,123.96949,117.784
5,participant_248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,350.044,350.044,426.16125,425.052,426.33398,425.221,426.51599,423.35,122.51147,119.307
6,participant_268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234.107,21.854,425.47461,321.988,425.74085,414.851,426.03382,303.129,122.42652,5.261
7,participant_270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,340.704,340.704,425.71912,424.641,426.18066,425.111,426.47302,423.312,122.5675,121.494
8,participant_278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234.431,234.431,426.58337,163.119,426.6919,390.302,426.73206,422.115,122.63293,101.655
9,participant_290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,193.616,193.616,426.18811,425.11,426.53271,421.287,426.80371,425.699,122.65356,121.576


In [ ]:
# Save result files
average_av_ratings.to_csv( generate_complete_path("average_av_ratings", subfolders=NOTEBOOK_NAME, file_extension=".csv") )
durations.to_csv( generate_complete_path("durations", subfolders=NOTEBOOK_NAME, file_extension=".csv") )